In [ ]:
###https://github.com/IBM/watson-machine-learning-samples/blob/master/cpd4.7/notebooks/python_sdk/lifecycle-management/Use%20scikit-learn%20and%20AI%20lifecycle%20capabilities%20to%20predict%20Boston%20house%20prices.ipynb
    
### deploy function - https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-deploy-functions_local.html?context=cpdaas 
## check this -- https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-deploy-py-function-write.html?context=cpdaas&audience=wdp

In [1]:
import gzip
with open('/project_data/data_asset/scoring_func 2.py', 'rb') as f_in, gzip.open('sffinal.gz', 'wb') as f_out:
    f_out.writelines(f_in)
    
    

In [15]:
from project_lib import Project
project = Project()
from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()

In [16]:
#project.save_data("scoring_func_inproj.tar.gz", data)
#asset = wslib.upload_file("sffinal.gz", "sffinal.gz")
#wslib.show(asset)


In [17]:
from json import loads
from IPython.display import JSON

In [18]:
%env USERNAME=admin
%env PASSWORD=uoKsHWi3obej
%env DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com

%env SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31
#db947328-7845-4f4b-a87c-2ca721960d74



env: USERNAME=admin
env: PASSWORD=uoKsHWi3obej
env: DATAPLATFORM_URL=https://cpd-wml.apps.neptune.cp.fyre.ibm.com
env: SPACE_ID=7b648ffc-53e3-4b16-8e59-3c55ef755f31


In [19]:
%env SPACE_ID  
#new space

'7b648ffc-53e3-4b16-8e59-3c55ef755f31'

In [20]:
wml_credentials = {
    "username": "admin",
    "passowrd": "uoKsHWi3obej",
    "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com",
    "instance_id": 'openshift',
    "version": '4.7'
}

In [21]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

client.set.default_space("1ad236d9-2b4d-4853-826f-358f58212ad2")
#db947328-7845-4f4b-a87c-2ca721960d74")

'SUCCESS'

#### get auth token

In [22]:
%%bash --out token

token=$(curl -sk -X POST \
        --header "Content-type: application/json" \
        -d "{\"username\":\"${USERNAME}\",\"password\":\"${PASSWORD}\"}" \
        "$DATAPLATFORM_URL/icp4d-api/v1/authorize")

token=${token#*token\":\"}
token=${token%%\"*}
echo $token

In [10]:
%env TOKEN=$token

env: TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6ImVZa2h5dGxMelFBcGt4ZUFxR3NIOGVaYTA3aXhqMXlzcVJzYTJLdnlHdncifQ.eyJ1c2VybmFtZSI6ImFkbWluIiwicm9sZSI6IkFkbWluIiwicGVybWlzc2lvbnMiOlsiY3JlYXRlX3Byb2plY3QiLCJtYW5hZ2VfY2F0YWxvZyIsIm1hbmFnZV92YXVsdHNfYW5kX3NlY3JldHMiLCJzaGFyZV9zZWNyZXRzIiwiYWRkX3ZhdWx0cyIsImFkbWluaXN0cmF0b3IiLCJjYW5fcHJvdmlzaW9uIiwibW9uaXRvcl9wbGF0Zm9ybSIsImNvbmZpZ3VyZV9wbGF0Zm9ybSIsInZpZXdfcGxhdGZvcm1faGVhbHRoIiwiY29uZmlndXJlX2F1dGgiLCJtYW5hZ2VfdXNlcnMiLCJtYW5hZ2VfZ3JvdXBzIiwibWFuYWdlX3NlcnZpY2VfaW5zdGFuY2VzIiwiY3JlYXRlX3NwYWNlIiwiYWNjZXNzX2NhdGFsb2ciXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJhZG1pbiIsImlzcyI6IktOT1hTU08iLCJhdWQiOiJEU1giLCJ1aWQiOiIxMDAwMzMwOTk5IiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJkaXNwbGF5X25hbWUiOiJhZG1pbiIsImFwaV9yZXF1ZXN0IjpmYWxzZSwiaWF0IjoxNjk4NzI1Mzg0LCJleHAiOjE2OTg3Njg1ODR9.bYfatLE9Nqr7FMuYYW2TXxUlQ5Y444HXCiSF0vmXe0nQZJZxelwaDLxwPCagU_ck5MIi51jwGBGbH3Ae_wjMj6pcy1Nyba-T2XF8tQs3I6-2iEXe1vasGzcSDO-5YqSfYKg0Ck4W2D_BijyAogHrrT4vqjFZy0QjsnIdOUxvU93wHf4ZskGdF

#### store model

In [23]:
%%bash --out model_payload

MODEL_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.2-py3.10"}}'
echo $MODEL_PAYLOAD

In [24]:
JSON(loads(model_payload))

<IPython.core.display.JSON object>

In [25]:
%env MODEL_PAYLOAD=$model_payload

env: MODEL_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "scikit_diabetes_model","description": "","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.2-py3.10"}}


In [340]:
%%bash --out model_id -s "$model_payload"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2023-10-25" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [341]:
%env MODEL_ID=$model_id

env: MODEL_ID=d535821f-a9b5-4772-b654-7f03114efd6c


#### store an external model

In [334]:
%%bash --out model_payload1

MODEL_PAYLOAD1='{"space_id": "'"$SPACE_ID"'","name": "scikit_diabetes_model","description": "This is description","type": "scikit-learn_1.1", "software_spec": {"name": "runtime-22.1-py3.10"}}'
echo $MODEL_PAYLOAD1


In [ ]:
%env MODEL_PAYLOAD1=$model_payload1

In [14]:
%%bash --out model_id1 -s "$MODEL_PAYLOAD1"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD1" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2023-10-24" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p

In [336]:
%env MODEL_ID=$model_id1

env: MODEL_ID=92157bc8-a81f-4109-9a59-5c1f3e5a4434


#### create a revision

In [342]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Initial model."}'
echo $REVISION_PAYLOAD

In [343]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Initial model."}


In [344]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Initial model.",
            "committed_at": "2023-10-30T17:33:44.002Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:32:23.353Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "resource_key": "db84066c-aa84-4213-b106-18a38fbe6809",
        "rev": "1",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### upload model content

In [346]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/diabetes/model/diabetes_model.tar.gz \
    -O diabetes_model.tar.gz 

In [348]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2020-08-01&content_format=native" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }'


In [349]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=85d85f4b-5776-4d26-93dc-c8e2875459b3


#### download a model

In [350]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --output "model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/download?space_id=$SPACE_ID&version=2020-08-01"

In [351]:
!ls -l model.tar.gz

-rw-rw----. 1 1000690000 wscommon 5322360 Oct 30 17:39 model.tar.gz


#### create deployment

In [352]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD

In [353]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD
JSON(loads(deployment_payload))


<IPython.core.display.JSON object>

In [354]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD
JSON(loads(deployment_payload))
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "d535821f-a9b5-4772-b654-7f03114efd6c"}}


In [355]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Diabetes Model deployment", "description": "This is description","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD
JSON(loads(deployment_payload))
%env DEPLOYMENT_PAYLOAD=$deployment_payload
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p
    


In [356]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=40458de1-f510-4fe4-856c-e29028a33de4


#### get deployment details

In [357]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2025-10-25" \
    | python -m json.tool


{
    "entity": {
        "asset": {
            "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes Model deployment",
        "online": {},
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
        "status": {
            "online_url": {
                "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            },
            "serving_urls": [
                "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            ],
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2023-10-30T17:39:59.586Z",
        "description": 

#### score

In [358]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}'
echo $SCORING_PAYLOAD

In [359]:
JSON(loads(scoring_payload))

<IPython.core.display.JSON object>

In [360]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={"space_id": "$SPACE_ID","input_data": [{"fields": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"], "values": [[-0.00188201652779104, -0.044641636506989, -0.0514740612388061, -0.0263278347173518, -0.00844872411121698, -0.019163339748222, 0.0744115640787594, -0.0394933828740919, -0.0683297436244215, -0.09220404962683], [0.0852989062966783, 0.0506801187398187, 0.0444512133365941, -0.00567061055493425, -0.0455994512826475, -0.0341944659141195, -0.0323559322397657, -0.00259226199818282, 0.00286377051894013, -0.0259303389894746]]}]}


In [361]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD"\
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-25" \
    | python -m json.tool

{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    82.3468335692293
                ],
                [
                    168.2544647515973
                ]
            ]
        }
    ]
}


#### list all deployments

In [362]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "resources": [
        {
            "entity": {
                "asset": {
                    "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
                },
                "custom": {},
                "deployed_asset_type": "model",
                "description": "This is description",
                "hardware_spec": {
                    "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
                    "name": "S",
                    "num_nodes": 1
                },
                "name": "Diabetes Model deployment",
                "online": {},
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
                "status": {
                    "online_url": {
                        "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
                    },
                    "serving_urls": [
                        "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458

#### model update

#### list the revisions

In [363]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/d535821f-a9b5-4772-b654-7f03114efd6c/revisions?space_id=7b648ffc-53e3-4b16-8e59-3c55ef755f31&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
                    "committed_at": "2023-10-30T17:33:44.002Z"
                },
                "created_at": "2023-10-30T17:32:23.353Z",
                "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
                "modified_at": "2023-10-30T17:32:23.353Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "resource_key": "db84066c-aa84-4213-b106-18a38fbe6809",
                "rev": "1",
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
            }
        }
    ]
}


#### create second revision

In [364]:
%%bash --out revision_payload

REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updated model."}'
echo $REVISION_PAYLOAD

In [366]:
JSON(loads(revision_payload))


<IPython.core.display.JSON object>

In [367]:
%env REVISION_PAYLOAD=$revision_payload

env: REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Updated model."}


In [368]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updated model.",
            "committed_at": "2023-10-30T17:41:02.002Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:38:52.745Z",
        "name": "scikit_diabetes_model",
        "owner": "1000330999",
        "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
        "rev": "2",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### update model metadata

In [369]:
%%bash --out update_payload

echo '[{"op": "add", "path": "/name", "value": "updated scikit model"}]'

In [370]:
%env UPDATE_PAYLOAD=$update_payload

env: UPDATE_PAYLOAD=[{"op": "add", "path": "/name", "value": "updated scikit model"}]


In [371]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$UPDATE_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool


{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2023-10-30T17:41:07.994Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:41:07.948Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### upload the new model content

In [372]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd4.0/models/scikit/diabetes/model/new_diabetes_model.tar.gz \
    -O new_diabetes_model.tar.gz 

In [373]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@new_diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/content?space_id=$SPACE_ID&version=2023-10-25&content_format=native" \
    | python -m json.tool

{
    "attachment_id": "c7b7bdb9-d30f-4ef0-a636-bbb4f0a3c2e1",
    "content_format": "native",
    "persisted": true
}


#### get model revisions 

In [374]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID/revisions?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "first": {
        "href": "/ml/v4/models/d535821f-a9b5-4772-b654-7f03114efd6c/revisions?space_id=7b648ffc-53e3-4b16-8e59-3c55ef755f31&limit=100"
    },
    "limit": 100,
    "resources": [
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Updated model.",
                    "committed_at": "2023-10-30T17:41:02.002Z"
                },
                "created_at": "2023-10-30T17:32:23.353Z",
                "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
                "modified_at": "2023-10-30T17:38:52.745Z",
                "name": "scikit_diabetes_model",
                "owner": "1000330999",
                "resource_key": "7600a9b5-e498-4fad-84af-9d781cda194b",
                "rev": "2",
                "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
            }
        },
        {
            "metadata": {
                "commit_info": {
                    "commit_message": "Initial model.",
             

In [90]:
#### get model details

In [375]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/models/$MODEL_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "hybrid_pipeline_software_specs": [],
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "scikit-learn_1.1"
    },
    "metadata": {
        "commit_info": {
            "committed_at": "2023-10-30T17:41:24.734Z"
        },
        "created_at": "2023-10-30T17:32:23.353Z",
        "id": "d535821f-a9b5-4772-b654-7f03114efd6c",
        "modified_at": "2023-10-30T17:41:17.582Z",
        "name": "updated scikit model",
        "owner": "1000330999",
        "resource_key": "ca9dc29d-52f7-4025-b560-aac25fd7a487",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### redeploy and score the new version

In [376]:
%%bash --out redeploy_payload

echo '[{"op": "replace", "path": "/asset", "value": {"id": "'"$MODEL_ID"'"}}]'

In [377]:
%env REDEPLOY_PAYLOAD=$redeploy_payload

env: REDEPLOY_PAYLOAD=[{"op": "replace", "path": "/asset", "value": {"id": "d535821f-a9b5-4772-b654-7f03114efd6c"}}]


In [378]:
%%bash

curl -sk -X PATCH \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$REDEPLOY_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2023-10-25" \
    | python -m json.tool

{
    "entity": {
        "asset": {
            "id": "d535821f-a9b5-4772-b654-7f03114efd6c"
        },
        "custom": {},
        "deployed_asset_type": "model",
        "description": "This is description",
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b",
            "name": "S",
            "num_nodes": 1
        },
        "name": "Diabetes Model deployment",
        "online": {},
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31",
        "status": {
            "online_url": {
                "url": "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            },
            "serving_urls": [
                "https://cpd-wml.apps.neptune.cp.fyre.ibm.com/ml/v4/deployments/40458de1-f510-4fe4-856c-e29028a33de4/predictions"
            ],
            "state": "ready"
        }
    },
    "metadata": {
        "created_at": "2023-10-30T17:39:59.586Z",
        "description": 

In [379]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-25" \
    | python -m json.tool


{
    "predictions": [
        {
            "fields": [
                "prediction"
            ],
            "values": [
                [
                    82.3468335692293
                ],
                [
                    168.2544647515973
                ]
            ]
        }
    ]
}


## Python Function

#### Define the function

In [433]:
## wrapper function 
def deployable_callable():
    """
    Deployable python function with score
    function implemented.
    """
    #try:
        #from statsmodels.stats.descriptivestats import describe
    n = 2
    #except ModuleNotFoundError as e:
        #print(f"statsmodels not installed: {str(e)}")
        
    def score(payload):
        """
        Score method.
        """
        try:
            data = payload['input_data'][0]['values']
            #print(data)
            return {
                'predictions': [
                    {'values': [i*n for i in data]}#, describe(data)]}
                ]
            }
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
        
    return score


In [434]:
import numpy as np

data = np.random.randint(2,20,2)
data_description = deployable_callable()({
    "input_data": [{
        "values" : [data]
    }]
})
print(data)
print(data_description["predictions"][0]["values"])

[2 4]
[array([4, 8])]


#### Create the function - add the function to WML repo 

In [26]:
%%bash --out scoring_payload

SCORING_PAYLOAD='{"name": "scaling python function2", "space_id": "'"$SPACE_ID"'", "software_spec": {"name": "runtime-22.2-py3.10"},"type": "python","sample_scoring_input": {  "input_data": [{ "values" : [12] }]}}'

echo $SCORING_PAYLOAD

 #{"name": "my-scoring-function", "space_id": "db947328-7845-4f4b-a87c-2ca721960d74", "software_spec": {"name": "runtime-22.2-py3.10"},"type": "python","sample_scoring_input": { "input_data": [{ "values" : ["12"] }]}}'\
       

In [27]:
JSON(loads(scoring_payload))

<IPython.core.display.JSON object>

In [28]:
%env SCORING_PAYLOAD=$scoring_payload

env: SCORING_PAYLOAD={"name": "scaling python function2", "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "software_spec": {"name": "runtime-22.2-py3.10"},"type": "python","sample_scoring_input": { "input_data": [{ "values" : [12] }]}}


In [29]:
%%bash --out function_id -s "$scoring_payload"


curl -sk -X POST  "$DATAPLATFORM_URL/ml/v4/functions?version=2023-10-26"\
        --header "Authorization: Bearer $TOKEN" \
        --header "Content-Type: application/json" \
        --header "Accept: application/json" \
        --data-raw "$SCORING_PAYLOAD" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 2p
                   

#### get function id

In [30]:
%env FUNCTION_ID=$function_id

env: FUNCTION_ID=bcc0ee29-3430-481a-a694-250b98894b5d


#### Create revision of the function

In [133]:
'''!wget -q https://github.com/hsanthan/qiskit-machine-learning/blob/main/scoring_func2.tar.gz -O scoring_func3.tar.gz '''

In [31]:
%%bash --out func_revision_payload

FUNC_REVISION_PAYLOAD='{"space_id": "'"$SPACE_ID"'", "commit_message": "Updating Score Function"}'
echo $FUNC_REVISION_PAYLOAD

In [32]:
%env FUNC_REVISION_PAYLOAD=$func_revision_payload

env: FUNC_REVISION_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31", "commit_message": "Updating Score Function"}


In [33]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$FUNC_REVISION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/revisions?version=2023-10-26" \
    | python -m json.tool

{
    "entity": {
        "sample_scoring_input": {
            "input_data": [
                {
                    "values": [
                        12
                    ]
                }
            ]
        },
        "software_spec": {
            "id": "b56101f1-309d-549b-a849-eaa63f77b2fb",
            "name": "runtime-22.2-py3.10"
        },
        "type": "python"
    },
    "metadata": {
        "commit_info": {
            "commit_message": "Updating Score Function",
            "committed_at": "2023-10-31T13:14:01.002Z"
        },
        "created_at": "2023-10-31T13:13:52.700Z",
        "id": "bcc0ee29-3430-481a-a694-250b98894b5d",
        "modified_at": "2023-10-31T13:13:52.700Z",
        "name": "scaling python function2",
        "owner": "1000330999",
        "rev": "1",
        "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
    },
    "system": {
        "warnings": []
    }
}


#### Upload the code to scoring function

In [124]:
'''!tar -zcvf score_fn.tar.gz /project_data/data_asset/my_scoring_func.py'''

tar: Removing leading `/' from member names
/project_data/data_asset/my_scoring_func.py


In [36]:
!wget -q https://raw.githubusercontent.com/hsanthan/qiskit-machine-learning/main/my_scoring_func.py -O score_fun_git.py

In [37]:
%%bash
cat score_fun_git.py

def score(payload):
        """
        Score method.
        """
        try:
            data = payload['input_data'][0]['values']
            # print(data)
            return {
                'predictions': [
                    {'values': [(data)*2]}  # , describe(data)]}
                ]
            }
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}



In [38]:
%%bash

gzip -c score_fun_git.py > score_fun_git.py.gz 

In [388]:
#### getting the python function file .py file from IBM COS

In [402]:
%env COS_CRN="fill out only if you want to create a new space"
%env COS_ENDPOINT=https://s3.us-east.cloud-object-storage.appdomain.cloud
%env COS_BUCKET=wml-cos
%env COS_ACCESS_KEY_ID=...
%env COS_SECRET_ACCESS_KEY=...
%env COS_API_KEY=lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn

%env AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token

env: COS_CRN="fill out only if you want to create a new space"
env: COS_ENDPOINT=https://s3.us-east.cloud-object-storage.appdomain.cloud
env: COS_BUCKET=wml-cos
env: COS_ACCESS_KEY_ID=...
env: COS_SECRET_ACCESS_KEY=...
env: COS_API_KEY=lh6naU4ptbQ-85U6suVxZqXRDT1tdYIBeu4I1aZk1gqn
env: AUTH_ENDPOINT=https://iam.cloud.ibm.com/oidc/token


In [403]:
%%bash --out cos_token

curl -s -X "POST" "$AUTH_ENDPOINT" \
     -H 'Accept: application/json' \
     -H 'Content-Type: application/x-www-form-urlencoded' \
     --data-urlencode "apikey=$COS_API_KEY" \
     --data-urlencode "response_type=cloud_iam" \
     --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
     | cut -d '"' -f 4

In [404]:
%env COS_TOKEN=$cos_token

env: COS_TOKEN=eyJraWQiOiIyMDIzMTAwODA4MzUiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLWM2ODQ0MTk3LWQ1Y2MtNDAwOS1hOTliLTQ3YjViMWYyMGFhOSIsImlkIjoiaWFtLVNlcnZpY2VJZC1jNjg0NDE5Ny1kNWNjLTQwMDktYTk5Yi00N2I1YjFmMjBhYTkiLCJyZWFsbWlkIjoiaWFtIiwianRpIjoiNTU0MTRiN2YtOTExMC00NzQzLWEwZGItYzM3ODFkOGE5MjBkIiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC1jNjg0NDE5Ny1kNWNjLTQwMDktYTk5Yi00N2I1YjFmMjBhYTkiLCJuYW1lIjoid21sLWNvcyIsInN1YiI6IlNlcnZpY2VJZC1jNjg0NDE5Ny1kNWNjLTQwMDktYTk5Yi00N2I1YjFmMjBhYTkiLCJzdWJfdHlwZSI6IlNlcnZpY2VJZCIsImF1dGhuIjp7InN1YiI6IlNlcnZpY2VJZC1jNjg0NDE5Ny1kNWNjLTQwMDktYTk5Yi00N2I1YjFmMjBhYTkiLCJpYW1faWQiOiJpYW0tU2VydmljZUlkLWM2ODQ0MTk3LWQ1Y2MtNDAwOS1hOTliLTQ3YjViMWYyMGFhOSIsInN1Yl90eXBlIjoiU2VydmljZUlkIiwibmFtZSI6IndtbC1jb3MifSwiYWNjb3VudCI6eyJ2YWxpZCI6dHJ1ZSwiYnNzIjoiMGE3NTI3NDVkNTA4NGFjOWIzOTBmNTYxZWNkZDcyOWQiLCJmcm96ZW4iOnRydWV9LCJpYXQiOjE2OTg2OTM5ODUsImV4cCI6MTY5ODY5NzU4NSwiaXNzIjoiaHR0cHM6Ly9pYW0uY2xvdWQuaWJtLmNvbS9vaWRjL3Rva2VuIiwiZ3JhbnRfdHlwZSI6InVybjppYm06cGFyYW1zOm9hdXRoOmdy

#### create .gz, deploy

In [405]:
%%bash
curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    "$COS_ENDPOINT/$COS_BUCKET"
    

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ListBucketResult xmlns="http://s3.amazonaws.com/doc/2006-03-01/"><Name>wml-cos</Name><Prefix></Prefix><Marker></Marker><MaxKeys>1000</MaxKeys><Delimiter></Delimiter><IsTruncated>false</IsTruncated><Contents><Key>my_dep_desc_func.py</Key><LastModified>2023-10-30T00:13:18.361Z</LastModified><ETag>&quot;17e1cd50a3edad7d1b3e8a5d2e616f22&quot;</ETag><Size>555</Size><Owner><ID>3910469f-f46c-426c-b842-e8e5ddbf216f</ID><DisplayName>3910469f-f46c-426c-b842-e8e5ddbf216f</DisplayName></Owner><StorageClass>STANDARD</StorageClass></Contents><Contents><Key>my_dep_desc_func_oct30.py</Key><LastModified>2023-10-30T19:25:10.147Z</LastModified><ETag>&quot;5f112a7cc4b147f3e0663f4d2366960d&quot;</ETag><Size>549</Size><Owner><ID>3910469f-f46c-426c-b842-e8e5ddbf216f</ID><DisplayName>3910469f-f46c-426c-b842-e8e5ddbf216f</DisplayName></Owner><StorageClass>STANDARD</StorageClass></Contents><Contents><Key>my_dep_func2.py</Key><LastModified>2023-10-29T23:24:

In [435]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "my_dep_func2.py" \
    "$COS_ENDPOINT/$COS_BUCKET/my_dep_func2.py"
    


In [ ]:
%%bash
cat scoringFunction.py

In [180]:
%%bash

gzip -c scoringFunction.py > scoringFunction.py.gz 

In [437]:
%%bash

gzip -c my_dep_func2.py > my_dep_func2.py.gz 


In [ ]:
ls -l sco*


In [283]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $COS_TOKEN" \
    --output "my_dep_desc_func.py" \
    "$COS_ENDPOINT/$COS_BUCKET/my_dep_desc_func.py"

In [284]:
%%bash
cat my_dep_desc_func.py

"""
Deployable python function with score
function implemented.
"""
# try:
from statsmodels.stats.descriptivestats import describe
n = 2

# except ModuleNotFoundError as e:
# print(f"statsmodels not installed: {str(e)}")

def score(payload):
    """
    Score method.
    """
    try:
        data = payload['input_data'][0]['values']
        # print(data)
        return {
            'predictions': [
                {'values': [describe(data)]}
            ]
        }
    except Exception as e:
        return {'predictions': [{'values': [repr(e)]}]}


In [285]:
%%bash

gzip -c my_dep_desc_func.py > my_dep_desc_func.py.gz 

In [ ]:
'''%%bash

wget https://github.com/IBM/watson-machine-learning-samples/raw/master/data/credit_risk/credit_risk_training_light.csv \
     -O credit_risk_training_light.csv
     
%%bash

tar -czvf model.pickle.tar.gz model.pickle

!ls -l model*'''

In [ ]:
'''%%bash
tar -czvf my_fn.py.tar.gz my_scoring_func.py'''

In [141]:
'''%%bash 

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "scoring_func_inproj.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26" '''

{
  "attachment_id": "71deeb58-98e2-494e-8c3c-011edbbb7ff7",
  "content_format": "native",
  "persisted": true
}

In [41]:
'''%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/x-compressed" \
    --data-binary "scoring_function.tgz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26" \
   | grep "attachment_id" | awk -F '"' '{ print $4 }'
'''

In [23]:
'''%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@diabetes_model.tar.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26"\
   | grep "attachment_id" | awk -F '"' '{ print $4 }'
'''


In [421]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@scoringFunction.py.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26"\
   | grep "attachment_id" | awk -F '"' '{ print $4 }'



In [489]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@my_dep_func2.py.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26"\
   | grep "attachment_id" | awk -F '"' '{ print $4 }'



In [39]:
%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/gzip" \
    --header "Accept: application/json" \
    --data-binary "@score_fun_git.py.gz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&version=2023-10-26"\
    | grep "attachment_id" | awk -F '"' '{ print $4 }'


In [40]:
%env ATTACHMENT_ID=$attachment_id

env: ATTACHMENT_ID=27e10622-43ed-46f5-9141-a4332d935783


In [ ]:
'''--header "Content-Encoding: gzip" \'''

In [44]:
'''from ibm_watson_studio_lib import access_project_or_space
wslib = access_project_or_space()


raw_data_1 = wslib.load_data('scoring_function.tar.gz')

raw_data_1
'''

In [ ]:
'''curl -X PUT 'https://us-south.ml.cloud.ibm.com/ml/v4/functions/:function_id/code?space_id=<string>&project_id=<string>&version=2020-09-01''''

In [63]:
'''%%bash --out attachment_id

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/x-gzip" \
    --data-binary "scoring_function.tgz" \
    "$DATAPLATFORM_URL/ml/v4/functions/$FUNCTION_ID/code?space_id=$SPACE_ID&catalog_id=075e3dbc-5022-4763-9b52-d255235c0fdc&version=2023-10-26" \
    | grep "attachment_id" | awk -F '"' '{ print $4 }''''

In [40]:
client.repository.list_functions()

------------------------------------  --------------------  ------------------------  ------  ----------  ----------------
GUID                                  NAME                  CREATED                   TYPE    SPEC_STATE  SPEC_REPLACEMENT
4a54575e-7415-418a-a58a-880bcaed6c59  py-scoring-function2  2023-10-27T01:40:43.002Z  python  supported
------------------------------------  --------------------  ------------------------  ------  ----------  ----------------


,GUID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT
0,4a54575e-7415-418a-a58a-880bcaed6c59,py-scoring-function2,2023-10-27T01:40:43.002Z,python,supported,


#### python function online deployment

In [41]:
%env FUNCTION_ID

'bcc0ee29-3430-481a-a694-250b98894b5d'

In [49]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "Py Function deployment", "description": "This is description","online": {},"asset": {"id": "'"$FUNCTION_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD

In [50]:
JSON(loads(deployment_payload))

<IPython.core.display.JSON object>

In [51]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

env: DEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "Py Function deployment", "description": "This is description","online": {},"asset": {"id": "bcc0ee29-3430-481a-a694-250b98894b5d"}}


In [52]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [53]:
%env DEPLOYMENT_ID=$deployment_id

env: DEPLOYMENT_ID=Deprecated


#### Score the function/prediction

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$SCORING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-30" \
    | python -m json.tool

In [54]:
%%bash

curl -sk -X POST "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2023-10-30"\
    --header 'Content-Type: application/json' \
    --header 'Accept: application/json' \
    --header "Authorization:Bearer  $TOKEN" \
    --data-raw '{"input_data":[{"values":[[3]]}]}' 


{"trace":"068181f4d070e0afc3a4049cb833c357","errors":[{"code":"deployment_does_not_exist","message":"Deployment with id 'e783ecb4-a3db-4e42-bd38-df33a8317902' does not exist. Re-try with a valid deployment id."}]}

#### python batch deployment

In [63]:
%%bash --out batch_deployment_payload

BATCH_DFEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "python batch func dep","description": "This is description","batch": {}, "hardware_spec": {"name": "S"},"asset": {"id": "'"$FUNCTION_ID"'"}}'
echo $BATCH_DFEPLOYMENT_PAYLOAD 

In [64]:
JSON(loads(batch_deployment_payload))

<IPython.core.display.JSON object>

In [65]:
%env BATCH_DFEPLOYMENT_PAYLOAD=$batch_deployment_payload

env: BATCH_DFEPLOYMENT_PAYLOAD={"space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","name": "python batch func dep","description": "This is description","batch": {}, "hardware_spec": {"name": "S"},"asset": {"id": "bcc0ee29-3430-481a-a694-250b98894b5d"}}


In [66]:
%%bash --out func_deployment_id_batch

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$BATCH_DFEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2023-10-30" | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p


In [67]:
%env FUNC_DEPLOYMENT_ID_BATCH=$func_deployment_id_batch

env: FUNC_DEPLOYMENT_ID_BATCH=7c0ddb07-49d6-4eeb-b1cc-01374d5ee989


#### Score the batch deployment

In [95]:
%%bash --out job_payload

JOB_PAYLOAD='{"name": "py func batch job", "space_id": "'"$SPACE_ID"'","deployment": {"id": "'"$FUNC_DEPLOYMENT_ID_BATCH"'"}, "hardware_spec": {"name": "S"},"scoring":{"input_data":[{"values":[[3]]}]}}'
echo $JOB_PAYLOAD 


In [96]:
JSON(loads(job_payload))

<IPython.core.display.JSON object>

In [97]:
%env JOB_PAYLOAD=$job_payload

env: JOB_PAYLOAD={"name": "py func batch job", "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31","deployment": {"id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"}, "hardware_spec": {"name": "S"},"scoring":{"input_data":[{"values":[[3]]}]}}


In [115]:
%%bash 

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$JOB_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?version=2023-10-30" 


{
  "entity": {
    "deployment": {
      "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
    },
    "hardware_spec": {
      "name": "S"
    },
    "scoring": {
      "input_data": [{
        "values": [[3]]
      }],
      "status": {
        "state": "queued"
      }
    }
  },
  "metadata": {
    "created_at": "2023-10-31T15:15:11.275Z",
    "id": "b7e8cdfd-0f25-4972-811b-0b37979aa389",
    "name": "py func batch job",
    "space_id": "7b648ffc-53e3-4b16-8e59-3c55ef755f31"
  }
}

In [ ]:
'''--out job_id
| grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p'''

In [116]:
#%env JOB_ID=$job_id

#### list all batch jobs

In [117]:
%%bash
curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs?space_id=$SPACE_ID&version=2020-08-01" 

{
  "resources": [{
    "entity": {
      "deployment": {
        "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
      },
      "hardware_spec": {
        "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b"
      },
      "platform_job": {
        "job_id": "58ac5da3-eda2-4130-854d-9c7fab86e4ad",
        "run_id": "5bfd3a7e-d458-42f4-8f16-7d10425686ff"
      },
      "scoring": {
        "input_data": [{
          "values": [[3]]
        }],
        "status": {
          "completed_at": "2023-10-31T15:17:55.072Z",
          "failure": {
            "trace": "9df53cddbbeb43c76f3d5f847bdc63a9",
            "errors": [{
              "code": "low_on_resources",
              "message": "Unable to run this job using the specified hardware specification due to insufficient server resources. Change to a smaller configuration or increase available server resources and try again.",
              "more_info": "The runtime failed to start due to 'insufficient resources'"
            }]
          },
     

In [120]:
%env JOB_ID=b7e8cdfd-0f25-4972-811b-0b37979aa389

env: JOB_ID=b7e8cdfd-0f25-4972-811b-0b37979aa389


In [121]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployment_jobs/$JOB_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

{
    "entity": {
        "deployment": {
            "id": "7c0ddb07-49d6-4eeb-b1cc-01374d5ee989"
        },
        "hardware_spec": {
            "id": "e7ed1d6c-2e89-42d7-aed5-863b972c1d2b"
        },
        "platform_job": {
            "job_id": "58ac5da3-eda2-4130-854d-9c7fab86e4ad",
            "run_id": "0e55f8d8-3877-46c6-8ac3-d9a5dfa395c7"
        },
        "scoring": {
            "input_data": [
                {
                    "values": [
                        [
                            3
                        ]
                    ]
                }
            ],
            "status": {
                "completed_at": "2023-10-31T15:18:50.521Z",
                "failure": {
                    "trace": "e2f3ba55cbcb6ba0c33a5794c1adccd9",
                    "errors": [
                        {
                            "code": "low_on_resources",
                            "message": "Unable to run this job using the specified hardware specification d